In [1]:
# 读取数据集，分析2018年的销售数据
import pandas as pd
import chardet
import xgboost
import numpy as np 
from sklearn.model_selection import train_test_split
from sklearn import metrics
import os
import re




def encode_text_dummy(df, name, delete =  False):
    dummies = pd.get_dummies(df[name])
    for x in dummies.columns:
        dummy_name = "{}-{}".format(name, x)
        df[dummy_name] = dummies[x]
    if delete:    
        df.drop(name, axis=1, inplace=True)


In [2]:
os.getcwd()

'/home/gang/GIX'

In [4]:
lst = os.listdir("KM")

In [5]:
lst

['销售.xlsx', '品类.xlsx', '店铺数量-周.xlsx']

In [7]:
lst[0]

'销售.xlsx'

In [11]:
df_sales = pd.read_excel("KM/"+lst[0])

In [12]:
df_sales.head(3)

,年,周,分公司,省份,城市,大类,中类,小类,销量,销额
0,2015,31,华中1分公司,湖北省,武汉市,服装,T恤,短袖T恤,50,4690.0
1,2015,31,华中1分公司,湖北省,武汉市,服装,牛仔裤,牛仔中裤,2,368.0
2,2015,31,华中1分公司,湖北省,武汉市,服装,牛仔裤,牛仔九分裤,1,129.0


In [16]:
df_sales["分公司"].value_counts().shape

(7,)

In [23]:
df_sales["年"].value_counts()

2018    215441
2017    208189
2016    158824
2019     28811
2015     19917
Name: 年, dtype: int64

In [24]:
group_year = df_sales.groupby(by = "年")
for key,item in group_year:
    group_year.get_group(key).to_csv("KM/"+str(key)+".csv",index = False)


In [25]:
df_2018 = pd.read_csv("KM/2018.csv",low_memory = False)

In [32]:
group=df_2018.groupby(by =["年","周","分公司","中类"]).sum()

In [36]:
group =group.reset_index()

In [52]:
group["-1周销量"] = np.ndarray(group.shape[0])
group["-2周销量"] = np.ndarray(group.shape[0])
group["-3周销量"] = np.ndarray(group.shape[0])
group["-1周销额"] = np.ndarray(group.shape[0])
group["-2周销额"] = np.ndarray(group.shape[0])
group["-3周销额"] = np.ndarray(group.shape[0])

In [73]:
df_sales=df_sales.groupby(by = ["年","周","分公司","中类"]).sum()
df_sales.reset_index(inplace=True)
group_zone = df_sales.groupby(by = "分公司")
for key,item in group_zone:
    data=group_zone.get_group(key)
    data.to_csv("KM/"+str(key)+".csv",index = False)
    group_cls = data.groupby(by = "中类")
    for key_,item in group_cls:
        group_cls.get_group(key_).to_csv("KM/sales/"+str(key)+str(key_)+".csv",index = False)
   

In [74]:
df_sample = pd.read_csv("KM/sales/华北分公司T恤.csv")

In [80]:
for i in range(3):
    df_sample["-"+str(i+1)+"周销量"] = np.ndarray(df_sample.shape[0])
    df_sample["-"+str(i+1)+"周销额"] = np.ndarray(df_sample.shape[0])

In [96]:
for year in df_sample["年"].value_counts().index:
    for week in df_sample["周"].value_counts().index:
        if (week >1):
            df_sample[(df_sample["周"]==week) & (df_sample["年"]==year)]["-1周销量"]= df_sample[(df_sample["周"]==(week-1)) & (df_sample["年"]==year)]["销量"]

/home/gang/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [119]:
df_sample[(df_sample["周"]==3) & (df_sample["年"]==2016)]["-1周销量"]=df_sample[(df_sample["周"]==2) & (df_sample["年"]==2016)]["销量"].values

/home/gang/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [125]:
df_sample.sort_values(by = ["年","周"],inplace = True)

In [129]:
df_sample.head(3)

,年,周,分公司,中类,销量,销额,-1周销量,-1周销额,-2周销量,-2周销额,-3周销量,-3周销额
0,2015,31,华北分公司,T恤,1776,87223.0000,6.898471e-310,6.898471e-310,6.898471e-310,6.898471e-310,6.898471e-310,6.898471e-310
1,2015,32,华北分公司,T恤,9069,474117.6900,4.643955e-310,6.898471e-310,6.898471e-310,6.898471e-310,6.898471e-310,6.898471e-310
2,2015,33,华北分公司,T恤,7618,407369.0573,4.643955e-310,4.643956e-310,4.643956e-310,4.643956e-310,4.643956e-310,4.643956e-310


In [136]:
df_sample["-1周销量"][1:] = df_sample["销量"][:-1]
df_sample["-1周销额"][1:] = df_sample["销额"][:-1]

/home/gang/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/gang/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [137]:
df_sample["-2周销量"][2:] = df_sample["销量"][:-2]
df_sample["-2周销额"][2:] = df_sample["销额"][:-2]

/home/gang/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/gang/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [138]:
df_sample["-3周销量"][3:] = df_sample["销量"][:-3]
df_sample["-3周销额"][3:] = df_sample["销额"][:-3]

/home/gang/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/gang/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [151]:
df_sample#.head(3)

,年,周,分公司,中类,销量,销额,-1周销额,-1周销量,-2周销额,-2周销量,-3周销额,-3周销量
0,2019,5,西北分公司,休闲鞋,1,231.43,2.00,2,2.0,2,2,2
1,2019,6,西北分公司,休闲鞋,1,199.50,231.43,1,2.0,2,2,2


In [149]:
for i in [-1,-2,-3,+1,+2,+3]:
    df_sample[str(i)+"周销额"]= df_sample.shape[0]
    df_sample[str(i)+"周销量"]= df_sample.shape[0]
    if(i<0):
        df_sample[str(i)+"周销量"][-i:] = df_sample["销量"][:i]
        df_sample[str(i)+"周销额"][-i:] = df_sample["销额"][:i]
    else:
        df_sample[str(i)+"周销量"][:-i] = df_sample["销量"][i:]
        df_sample[str(i)+"周销额"][:-i] = df_sample["销额"][i:]

/home/gang/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/home/gang/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


ValueError: cannot set using a slice indexer with a different length than the value

In [145]:
import os
ls = os.listdir("KM/sales")
ls

['总部棉衣.csv',
 '华北分公司T恤.csv',
 '西北分公司衬衫.csv',
 '华中1分公司衬衫.csv',
 '西北分公司运动鞋.csv',
 '华南分公司牛仔裤.csv',
 '总部线衣.csv',
 '华中2分公司休闲鞋.csv',
 '西南分公司靴子.csv',
 '西南分公司皮鞋.csv',
 '华中2分公司靴子.csv',
 '华南分公司羽绒服.csv',
 '华中1分公司牛仔裤.csv',
 '华中2分公司牛仔裤.csv',
 '华南分公司连衣裙.csv',
 '西北分公司裤装.csv',
 '华北分公司羽绒服.csv',
 '华北分公司板鞋.csv',
 '西南分公司拖鞋.csv',
 '西南分公司夹克.csv',
 '华中1分公司风衣.csv',
 '西南分公司裤装.csv',
 '总部服饰配件.csv',
 '华中2分公司皮衣.csv',
 '总部皮鞋.csv',
 '华南分公司板鞋.csv',
 '总部运动鞋.csv',
 '华南分公司卫衣.csv',
 '总部休闲鞋.csv',
 '华北分公司运动鞋.csv',
 '华中1分公司靴子.csv',
 '华南分公司T恤.csv',
 '西南分公司衬衫.csv',
 '华南分公司运动鞋.csv',
 '华北分公司线衣.csv',
 '西北分公司服饰配件.csv',
 '华北分公司服饰配件.csv',
 '西北分公司风衣.csv',
 '西北分公司大衣.csv',
 '华中2分公司衬衫.csv',
 '西北分公司线衣.csv',
 '华北分公司夹克.csv',
 '西北分公司其他外套.csv',
 '总部皮衣.csv',
 '华中2分公司板鞋.csv',
 '西北分公司T恤.csv',
 '西北分公司拖鞋.csv',
 '华中1分公司其他外套.csv',
 '华南分公司其他外套.csv',
 '西南分公司其他外套.csv',
 '华中2分公司其他外套.csv',
 '华北分公司牛仔裤.csv',
 '华南分公司衬衫.csv',
 '总部夹克.csv',
 '西北分公司卫衣.csv',
 '总部羽绒服.csv',
 '西北分公司休闲鞋.csv',
 '总部裤装.csv',
 '华南分公司棉衣.csv',
 '总部风衣.csv',
 '华中2分公司拖鞋.csv',
 '西南分公司皮衣.csv

In [152]:
for file in ls:
    df_sample = pd.read_csv("KM/sales/"+file)
    df_sample.sort_values(by = ["年","周"],inplace = True)
    if(df_sample.shape[0]<5):
        continue
    for i in [-1,-2,-3,+1,+2,+3]:
        df_sample[str(i)+"周销额"]= df_sample.shape[0]
        df_sample[str(i)+"周销量"]= df_sample.shape[0]
        if(i<0):
            df_sample[str(i)+"周销量"][-i:] = df_sample["销量"][:i]
            df_sample[str(i)+"周销额"][-i:] = df_sample["销额"][:i]
        else:
            df_sample[str(i)+"周销量"][:-i] = df_sample["销量"][i:]
            df_sample[str(i)+"周销额"][:-i] = df_sample["销额"][i:]
        df_sample.to_csv("KM/sales/"+file,index = False)    
            
    

/home/gang/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
/home/gang/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
/home/gang/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/ho

In [153]:
df1 = pd.read_csv("KM/sales/"+ls[1])
df2 = pd.read_csv("KM/sales/"+ls[2])

In [159]:
short=[]
for file in ls:
    df_sample = pd.read_csv("KM/sales/"+file)
    
    if(df_sample.shape[0]<5):
        short.append(file)
        print(file) 

华中2分公司休闲鞋.csv
总部休闲鞋.csv
西北分公司休闲鞋.csv
总部半裙.csv
华中1分公司半裙.csv


In [161]:
df = pd.DataFrame()
for file in ls:
    if(file in short):
        continue
    df_sample = pd.read_csv("KM/sales/"+file)
    df = pd.concat([df,df_sample])
    

In [168]:
df[df["年"]==2019]["周"].value_counts()

2    130
3    129
1    127
4    126
5    124
6    109
7     98
Name: 周, dtype: int64

In [185]:
df_ = df[-((df["年"]==2015)&(df["周"]<34))]

In [186]:
df_ = df_[-((df_["年"]==2019)&(df_["周"]>4))]

In [194]:
df_.to_csv("KM_pre.csv",index = False)

In [193]:
df_.head(3)

,年,周,分公司,中类,销量,销额,-1周销额,-1周销量,-2周销额,-2周销量,-3周销额,-3周销量,1周销额,1周销量,2周销额,2周销量,3周销额,3周销量,未来4周销量之和,未来4周销额之和
0,2017,42,总部,棉衣,3,210.0,22.0,22,22.0,22,22.0,22,6500.0,65,800.0,8,4600.0,46,122,12110.0
1,2017,45,总部,棉衣,65,6500.0,210.0,3,22.0,22,22.0,22,800.0,8,4600.0,46,24020.0,251,370,35920.0
2,2017,46,总部,棉衣,8,800.0,6500.0,65,210.0,3,22.0,22,4600.0,46,24020.0,251,6660.0,67,372,36080.0


In [192]:
df_["未来4周销量之和"] = df_["销量"]+ df_["1周销量"]+ df_["2周销量"]+ df_["3周销量"]
df_["未来4周销额之和"] = df_["销额"]+ df_["1周销额"]+ df_["2周销额"]+ df_["3周销额"]

In [206]:
df_.columns[:]

Index(['年', '周', '分公司', '中类', '销量', '销额', '-1周销额', '-1周销量', '-2周销额', '-2周销量',
       '-3周销额', '-3周销量', '1周销额', '1周销量', '2周销额', '2周销量', '3周销额', '3周销量',
       '未来4周销量之和', '未来4周销额之和'],
      dtype='object')

In [208]:
df_.iloc[:,0:12]

,年,周,分公司,中类,销量,销额,-1周销额,-1周销量,-2周销额,-2周销量,-3周销额,-3周销量
0,2017,42,总部,棉衣,3,210.0000,22.0000,22,22.0000,22,22.0000,22
1,2017,45,总部,棉衣,65,6500.0000,210.0000,3,22.0000,22,22.0000,22
2,2017,46,总部,棉衣,8,800.0000,6500.0000,65,210.0000,3,22.0000,22
3,2018,2,总部,棉衣,46,4600.0000,800.0000,8,6500.0000,65,210.0000,3
4,2018,3,总部,棉衣,251,24020.0000,4600.0000,46,800.0000,8,6500.0000,65
5,2018,4,总部,棉衣,67,6660.0000,24020.0000,251,4600.0000,46,800.0000,8
6,2018,5,总部,棉衣,455,40220.0000,6660.0000,67,24020.0000,251,4600.0000,46
7,2018,6,总部,棉衣,31,3100.0000,40220.0000,455,6660.0000,67,24020.0000,251
8,2018,44,总部,棉衣,8,1684.5700,3100.0000,31,40220.0000,455,6660.0000,67
9,2018,45,总部,棉衣,533,30429.0100,1684.5700,8,3100.0000,31,40220.0000,455


In [211]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import train_test_split

In [207]:
x = df_.iloc[:,0:12].drop(["销额","销量"],axis = 1)

In [210]:
y = df_[['销量', '销额','1周销额', '1周销量','未来4周销量之和', '未来4周销额之和']]

In [216]:
x.columns

Index(['年', '周', '分公司', '中类', '-1周销额', '-1周销量', '-2周销额', '-2周销量', '-3周销额',
       '-3周销量'],
      dtype='object')

In [222]:
encode_text_dummy(x,name = "分公司",delete = True)

In [223]:
encode_text_dummy(x,name = "中类",delete = True)

In [224]:
x.head(3)

,年,周,-1周销额,-1周销量,-2周销额,-2周销量,-3周销额,-3周销量,分公司-华中1分公司,分公司-华中2分公司,...,中类-皮衣,中类-皮鞋,中类-线衣,中类-羽绒服,中类-衬衫,中类-裤装,中类-运动鞋,中类-连衣裙,中类-靴子,中类-风衣
0,2017,42,22.0,22,22.0,22,22.0,22,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2017,45,210.0,3,22.0,22,22.0,22,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2017,46,6500.0,65,210.0,3,22.0,22,0,0,...,0,0,0,0,0,0,0,0,0,0


In [225]:
train_x,test_x,train_y,test_y = train_test_split(x,y,train_size = 0.8)

/home/gang/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [299]:
regr_multirf = MultiOutputRegressor(RandomForestRegressor(max_depth=10, random_state=24))
regr_multirf.fit(train_x, train_y)

MultiOutputRegressor(estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=10,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=24, verbose=0, warm_start=False),
           n_jobs=1)

In [300]:
pred_y = regr_multirf.predict(test_x)

In [237]:
pred_y.shape

(3538, 6)

In [246]:
pred_y[:,1].shape

(3538,)

In [241]:
test_y.values.shape

(3538, 6)

In [301]:
for i in range(6):
    error = abs(pred_y[:,i]-y_test[:,i])
    scaled = (error/pred_y[:,i])*100
    score = scaled.mean()
    print(score)

41.25899394095397
43.08468247001567
54.46140275124156
56.34780735688484
52.629897471777866
53.57584030636756


误差

In [294]:
(pred_y==0).sum()

0

In [283]:
y_test = test_y.values

In [282]:
pred_y.dtype

dtype('float64')

In [286]:
pred_y=pred_y.astype("float64")
y_test=y_test.astype("float64")